In [2]:
import os
from glob import glob
import numpy as np
import re
import matplotlib as plt 
%matplotlib

ImportError: dlopen(/Users/lillyel-said/anaconda2/lib/python2.7/site-packages/matplotlib/ft2font.so, 2): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/lillyel-said/anaconda2/lib/libfreetype.6.dylib
  Reason: Incompatible library version: libfreetype.6.dylib requires version 45.0.0 or later, but libpng16.16.dylib provides version 39.0.0

In [3]:
#define paths

main_dir='/Volumes/daelsaid/data/emoperception_processing/17131'
behav_dir='1/data/emoperception_processing/happy_cleanbehav/temp/'
#behav_dir='/Volumes/daelsaid/data/emoperception_processing/17131'
#happy=glob(os.path.join(main_dir,'emo_2','*.log'))[0]
#anger=glob(os.path.join(main_dir,'emo_1','*.log'))[0]

files=glob(os.path.join(behav_dir,'*.log'))

#create filename to be used for txt file of conditions


filename=happy.split('/')[-1].split('.')[0]
subj=filename.split('_')[0]

print filename,subj
# for f in files:
#     all_split=(re.split('; |_|/*|',f))
#     filename.append([all_split[10]+'_'+all_split[12]+'_'])


NameError: name 'happy' is not defined

In [ ]:
#read subject log lines into array

def emotion_lines(emotion_files):
    subj_log_lines=np.genfromtxt(emotion_files,delimiter="\t",dtype=str)
    return subj_log_lines



subj_log_lines=np.genfromtxt(happy,delimiter="\t",dtype=str)

In [ ]:
#get scan trigger and trial timings and different trials

trials=[]
trial_timing=[]
scan_start=[]

for idx,line in enumerate(subj_log_lines):
    if line[2] == 'text_2: autoDraw = True':
        scan_start=float(line[0])        
    if line[2].endswith('autoDraw = True'):
        trial_timing.append([line[0],line[1],line[2]])
    if line[2].startswith('New trial'):
        trials.append([line[0],line[2]])
    

In [ ]:
#extract duration of stimulus in seconds

fix=np.array([x for x in trial_timing if x[2].startswith('FixCross')])
image=np.array([x for x in trial_timing if x[2].startswith('Face')])
mask=np.array([x for x in trial_timing if x[2].startswith('Mask')])


f=fix[:,:][:,0].reshape(150,1).astype(float) 
i=image[:,:][:,0].reshape(150,1).astype(float)
m=mask[:,:][:,0].reshape(150,1).astype(float) 

image_timing=np.subtract(m,i)
iti=np.subtract(f[1:],f[:-1])


In [ ]:
#determine duration adn onsent time relative to scan start 
image_relative=np.subtract(i,scan_start)
mask_relative=np.subtract(m,scan_start)

In [ ]:
#extract condition per trial, and percent emotion  of stimulus
emotion_with_percent=[]
index_with_cor=[]
for_mat_file=[]
for ix,trial in enumerate(trials):
    index_with_cor.append([ix,trial[1].split(':')[3].split("['")[1].split("']")[0]])
    percent_emotion=trial[1].split('{')[:][1].split(',')[4]
    emotion=trial[1].split('{')[:][1].split(',')[0]
    combined=(emotion.split("': u'")[1]+'_'+percent_emotion.split(': ')[1]).replace("'",'')
    emotion_with_percent.append([ix,combined])
    for_mat_file.append(combined)


In [ ]:
for ix, image in enumerate(trials):
    if 'bmp' in image[-1]:
        print image

In [ ]:
#combine arrays to get 3 column array


arr=np.append(image_timing,emotion_with_percent,axis=1)
arr=np.append(image_relative,arr,axis=1)
print np.delete(arr,2,axis=1)
arr=np.delete(arr,2,axis=1)
arr=np.delete(arr,0,axis=0)



In [ ]:
#break apart array by condition/percent emotion

happy_0=[]
happy_25=[]
happy_50=[]
happy_75=[]
happy_100=[]

for index,emo in enumerate(arr):
    if '_0' in emo[:][2:][0]:
        happy_0.append(emo)
    if '_25' in emo[:][2:][0]:
        happy_25.append(emo)
    if '_50' in emo[:][2:][0]:
        happy_50.append(emo)
    if '_75' in emo[:][2:][0]:
        happy_75.append(emo)
        print happy_75
    if '_100' in emo[:][2:][0]:
        happy_100.append(emo)        

In [ ]:
#generate mat file of each condition adn regressors

# import scipy.io as scp
scp.savemat('test.mat',dict({'duration':image_timing,'onsets':image_relative,'names':['happy','anger']}))
# arr.getfield()

In [ ]:
#write array to text file

def write_arr_tofile(array, file_with_ext,savemat):
    with open(file_with_ext,"w+") as myfile:
        np.savetxt(file_with_ext,array,delimiter=' ',fmt='%s')
    

In [ ]:
# write_arr_tofile(happy_0,os.path.join(behav_dir,filename+'_happy0.txt'),1)
# write_arr_tofile(happy_25,os.path.join(behav_dir,filename+'_happy25.txt'),1)
# write_arr_tofile(happy_50,os.path.join(behav_dir,filename+'_happy50.txt'),1)
# write_arr_tofile(happy_75,os.path.join(behav_dir,filename+'_happy75.txt'),1)
# write_arr_tofile(happy_100,os.path.join(behav_dir,filename+'_happy100.txt'),1)

In [ ]:

import scipy.io as scp

print arr[0:][0:]
            
scp.savemat(os.path.join(behav_dir,'output',subj+'_happy0.txt'),happy_0,appendmat=True)

In [ ]:
print subj_log_lines
actual=[]
for idx,line in enumerate(subj_log_lines):
    if 'theseKeys' in line:
        print line
    if line[1].startswith('DATA'):
        actual.append(line[2].split(': ')[1])


In [ ]:
response_corr=[]

for y in range(len(index_with_cor)):
    print y
#     print [index_with_cor[x][0],index_with_cor[x][1]==actual[x]]
#     response_corr.append([index_with_cor[x][0],index_with_cor[x][1]==actual[x]])


In [ ]:
# print 'face --> mask:',(np.subtract(float(fix[0][0]),float(mask[0][0])))
# print 'mask --> fixation:',(np.subtract(float(fix[0][0]),float(image[0][0])))
# print 'ISI, mask --> image:',(np.subtract(float(mask[0][0]),float(image[0][0])))
# print 'number of trials:', len(trials)

In [ ]:
def extract_emoperception_conditions(behav_data_path,file,output_txt_path):
    behav_dir=behav_data_path
    output_dir=output_txt_path
    filename=file.split('/')[-1]

    #define paths
    # file=glob(os.path.join(behav_dir,'*.log'))[2]

    #read subject log lines

    subj_log_lines=np.genfromtxt(file,delimiter="\t",dtype=str)

    #get scan trigger and trial timings and different trials

    trials=[]
    trial_timing=[]
    scan_start=[]

    for idx,line in enumerate(subj_log_lines):
        if line[2] == 'text_2: autoDraw = True':
            scan_start=float(line[0])
        if line[2].endswith('autoDraw = True'):
            trial_timing.append([line[0],line[1],line[2]])
        if line[2].startswith('New trial'):
            trials.append([line[0],line[2]])

    #extract duration of stimulus in seconds

    fix=np.array([x for x in trial_timing if x[2].startswith('FixCross')])
    image=np.array([x for x in trial_timing if x[2].startswith('Face')])
    mask=np.array([x for x in trial_timing if x[2].startswith('Mask')])


    f=fix[:,:][:,0].reshape(150,1).astype(float)
    i=image[:,:][:,0].reshape(150,1).astype(float)
    m=mask[:,:][:,0].reshape(150,1).astype(float)

    image_timing=np.subtract(m,i)
    iti=np.subtract(f[1:],f[:-1])

    #determine duration adn onsent time relative to scan start
    image_relative=np.subtract(i,scan_start)
    mask_relative=np.subtract(m,scan_start)

    #extract condition per trial, and percent emotion  of stimulus
    emotion_with_percent=[]
    index_with_cor=[]

    for ix,trial in enumerate(trials):
        index_with_cor.append([ix,trial[1].split(':')[3].split("['")[1].split("']")[0]])
        percent_emotion=trial[1].split('{')[:][1].split(',')[4]
        emotion=trial[1].split('{')[:][1].split(',')[0]
        combined=(emotion.split("': u'")[1]+'_'+percent_emotion.split(': ')[1]).replace("'",'')
        emotion_with_percent.append([ix,combined])
        #combine arrays to get 3 column array


    #create new array with 3 columns
    arr=np.append(image_timing,emotion_with_percent,axis=1)
    arr=np.append(image_relative,arr,axis=1)
    # print np.delete(arr,2,axis=1)
    arr=np.delete(arr,2,axis=1)
    arr=np.delete(arr,0,axis=0)


    #break apart array by condition/percent emotion

    happy_0=[]
    happy_25=[]
    happy_50=[]
    happy_75=[]
    happy_100=[]

    for index,emo in enumerate(arr):
        if '_0' in emo[:][2:][0]:
            happy_0.append(emo)
        if '_25' in emo[:][2:][0]:
            happy_25.append(emo)
        if '_50' in emo[:][2:][0]:
            happy_50.append(emo)
        if '_75' in emo[:][2:][0]:
            happy_75.append(emo)
        if '_100' in emo[:][2:][0]:
            happy_100.append(emo)

    #write array to text file

    def write_arr_tofile(array, file_with_ext):
        with open(file_with_ext,"w+") as myfile:
            np.savetxt(file_with_ext,array,delimiter=' ',fmt='%s')



    write_arr_tofile(happy_0,os.path.join(behav_dir,output_dir,filename+'_happy0.txt'))
    write_arr_tofile(happy_25,os.path.join(behav_dir,output_dir,filename+'_happy25.txt'))
    write_arr_tofile(happy_50,os.path.join(behav_dir,output_dir,filename+'_happy50.txt'))
    write_arr_tofile(happy_75,os.path.join(behav_dir,output_dir,filename+'_happy75.txt'))
    write_arr_tofile(happy_100,os.path.join(behav_dir,output_dir,filename+'_happy100.txt'))

    actual=[]
    for idx,line in enumerate(subj_log_lines):
        if 'theseKeys' in line[2]:
            print line[2]
        if line[1].startswith('DATA'):
            actual.append(line[2].split(': ')[1])

    response_corr=[]

    for y in range(len(index_with_cor)):
        print y
    #     print [index_with_cor[x][0],index_with_cor[x][1]==actual[x]]
    #     response_corr.append([index_with_cor[x][0],index_with_cor[x][1]==actual[x]])



In [ ]:
# behav_path=('/Users/lillyel-said/Desktop/stanford/scripts/jupyter_notebooks/anger_good')
# curr_dir = os.getcwd()
# outputpath=os.path.join(behav_path,'output')

# current_dir=[f for f in os.listdir('.') if os.path.isfile(f)]

# for f in current_dir:
#     filename, ext = os.path.splitext(f)
#     if ext=='.log':
#         logfile = os.path.join(curr_dir,f)
#         try:
#             extract_emoperception_conditions(behav_path,logfile,outputpath)
#         except:
#             pass
